In [1]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
import os
from rasterio.coords import disjoint_bounds
from rasterio.transform import from_origin
from rasterio.windows import Window
from collections import Counter
import pandas as pd

## Procedure
CRS

RESAMPLE

HAZARD CLASSIFICATION

EAR RECLASSIFICATION IF REQUIRED

FIND INTERSECTING PART AND MANAGE EXTENT

## RiskChanges Library


In [2]:
def reclassify(in_image, out_image, base, stepsize, maxval):
    input_image = rasterio.open(in_image)
    intensity_data = input_image.read(1)

    prev = base
    thresholds = np.arange(start=base, stop=maxval+1, step=stepsize).tolist()
    intensity_data[intensity_data < base] = input_image.nodata
    intensity_data_classified = np.copy(intensity_data)

    for i, threshold in enumerate(thresholds):
        #mean=intensity_data[((intensity_data<threshold) & (intensity_data>=prev))].mean()

        intensity_data_classified[(
            (intensity_data < threshold) & (intensity_data >= prev))] = i

        prev = threshold

        # if it is the last value, need to assign the max class for all result
        if threshold == thresholds[-1]:
            intensity_data_classified[(
                intensity_data >= thresholds[-1])] = i
    # print(intensity_data_classified)

    with rasterio.Env():
        profile = input_image.profile
        with rasterio.open(out_image, 'w', **profile) as dst:
            dst.write(intensity_data_classified, 1)
        dst = None
    input_image = None
    
def ClassifyHazard(hazard_file, base, stepsize, threshold):
    infile = hazard_file
    outfile = hazard_file.replace(".tif", "_reclassified.tif")
    if os.path.isfile(outfile):
        pass
    else:
        reclassify(infile, outfile, base, stepsize, threshold)
    return outfile


def readhaz(hazfile_path, base,step_size,threshold):
    outfile = ClassifyHazard(hazfile_path, base, step_size, threshold)
    src = rasterio.open(outfile)
    return src

## Execute Raster Projection
Refer to raster_proj.ipynb 

## Re-assign pixel value

In [ ]:
input_file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/LandCover_ROU_UTM.tif"
output_file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/LandCover_ROU_UTM_classified.tif"

# input_file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_AUS_UTM.tif"
# output_file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_AUS_UTM_classified.tif"

In [ ]:
reassign_value_dict={
    111:1,112:1,121:1,122:1,123:1,124:1,131:1,132:1,133:1,141:1,142:1,
    211:2,212:2,213:2,221:2,222:2,223:2,231:2,241:2,242:2,243:2,244:2,
    311:3,312:3,313:3,321:3,322:3,323:3,324:3,331:3,332:3,333:3,334:3,335:3,
    411:4,412:4,421:4,422:4,423:4,511:5,512:5,521:5,522:5,523:5}
# source:https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_CORINE_V20_100m#bands
#1=artificial surface
#2=agriculture
#3=forest
#4=wetland
#5=waterbodies

In [ ]:
ear_raster=rasterio.open(input_file_path)
src_crs=ear_raster.crs
no_of_bands=ear_raster.count
print(f"no of band: {no_of_bands}")

In [ ]:
ear_raster_band_1=ear_raster.read(1)
unique_band_values=np.unique(ear_raster_band_1)
print("Unique pixel values in band 1:", unique_band_values)

In [ ]:
for key in reassign_value_dict.keys():
    ear_raster_band_1[ear_raster_band_1==key]=reassign_value_dict[key]
unique_band_values_2=np.unique(ear_raster_band_1)
print("Unique pixel values in band 1:", unique_band_values_2)


In [ ]:
height, width = ear_raster_band_1.shape
# print(f"height:{height},width:{width}")
meta = {
    'driver': 'GTiff',
    'dtype': ear_raster_band_1.dtype,
    'count': 1,  # Number of bands
    'height': height,
    'width': width,
    'crs': ear_raster.crs,  # Replace with your desired CRS
    'transform': ear_raster.transform
}

with rasterio.open(output_file_path, 'w', **meta) as dst:
    dst.write(ear_raster_band_1, 1)

## Hazard resample and classification

In [3]:
import rasterio
from rasterio.enums import Resampling

hazard_input_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_HAZ_DUMMY_AUS_UTM.tif"
ear_input_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_AUS_UTM.tif"

resampled_haz_output_file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_HAZ_DUMMY_AUS_UTM_resampled.tif"
resampled_ear_output_file_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure_AUS/output/LC_AUS_UTM_resampled.tif"

with rasterio.open(ear_input_path) as ear_raster, rasterio.open(hazard_input_path) as hazard_raster:
    final_x_res=int(ear_raster.res[0])
    final_y_res=int(ear_raster.res[1])
    print(final_x_res,final_y_res,"x y resolution")
    
    resampled_haz_width=int((hazard_raster.width * hazard_raster.res[0]) / final_x_res)
    resampled_haz_height=int((hazard_raster.height * hazard_raster.res[1]) / final_y_res)

    resampled_ear_width=int((ear_raster.width * ear_raster.res[0]) / final_x_res)
    resampled_ear_height=int((ear_raster.height * ear_raster.res[1]) / final_y_res)
   
    haz_x_origin, haz_y_origin = hazard_raster.transform[2],hazard_raster.transform[5] 
    ear_x_origin, ear_y_origin = ear_raster.transform[2],ear_raster.transform[5] 

    # Create a new Affine transformation with the updated pixel dimensions
    new_haz_transform = rasterio.transform.Affine(final_x_res,0.0, haz_x_origin,0.0,-final_y_res, haz_y_origin)
    new_ear_transform = rasterio.transform.Affine(final_x_res,0.0, ear_x_origin,0.0,-final_y_res, ear_y_origin)
 
    resampled_haz_data = hazard_raster.read(
            out_shape=(hazard_raster.count, resampled_haz_height, resampled_haz_width),
            resampling=Resampling.nearest
        )
    resampled_ear_data = ear_raster.read(
            out_shape=(ear_raster.count, resampled_ear_height, resampled_ear_width),
            resampling=Resampling.nearest
        )

    resampled_haz_meta = {
        'driver': 'GTiff',
        'dtype': resampled_haz_data.dtype,
        'count': 1,  # Number of bands
        'height': resampled_haz_height,
        'width': resampled_haz_width,
        'crs': hazard_raster.crs,  # Replace with your desired CRS
        'transform': new_haz_transform
    }
    resampled_ear_meta = {
        'driver': 'GTiff',
        'dtype': resampled_ear_data.dtype,
        'count': 1,  # Number of bands
        'height': resampled_ear_height,
        'width': resampled_ear_width,
        'crs': ear_raster.crs,  # Replace with your desired CRS
        'transform': new_ear_transform
    }
    with rasterio.open(resampled_ear_output_file_path, 'w', **resampled_ear_meta) as dst:
        dst.write(resampled_ear_data)
    with rasterio.open(resampled_haz_output_file_path, 'w', **resampled_haz_meta) as dst:
        dst.write(resampled_haz_data)
    # print(resampled_data)

82 82 x y resolution


In [ ]:
haz_in_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/FL_ROU_600_UTM_resampled.tif"
# with rasterio.open(haz_in_path) as src:
#     band1 = src.read(1)

#     min_value = np.min(band1)
#     max_value = np.max(band1)
#     unique_value = np.unique(band1)

#     print(f"Minimum value: {min_value}")
#     print(f"Maximum value: {max_value}")
#     print(f"Unique value: {unique_value}")
reclassified_hazard=readhaz(haz_in_path,0,2,20)

## Making same extent, origin and  number of pixels

In [ ]:
hazard_input_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/FL_ROU_600_UTM_resampled_reclassified.tif"
ear_input_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/LandCover_ROU_UTM_classified_resampled.tif"

final_haz_out_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/FL_ROU_600_UTM_resampled_reclassified_ready.tif"
final_ear_out_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/LandCover_ROU_UTM_classified_resampled_ready.tif"

In [ ]:
with rasterio.open(hazard_input_path) as hazard_raster, rasterio.open(ear_input_path) as ear_raster:
    hazard_raster_data = hazard_raster.read(1)  # Read the first band of raster 1
    ear_raster_data = ear_raster.read(1)  # Read the first band of raster 2
    hazard_bounds = hazard_raster.bounds
    ear_bounds = ear_raster.bounds
    hazard_transform = hazard_raster.transform
    ear_transform = ear_raster.transform
    hazard_raster_profile = hazard_raster.profile
    ear_raster_profile = ear_raster.profile

# Calculate intersection bounds
intersection_bounds = (
    max(ear_bounds.left, hazard_bounds.left),
    max(ear_bounds.bottom, hazard_bounds.bottom),
    min(ear_bounds.right, hazard_bounds.right),
    min(ear_bounds.top, hazard_bounds.top)
)


In [ ]:
with rasterio.open(hazard_input_path) as hazard_raster, rasterio.open(ear_input_path) as ear_raster:
    hazard_raster_data = hazard_raster.read(1)  # Read the first band of raster 1
    ear_raster_data = ear_raster.read(1)  # Read the first band of raster 2
    hazard_bounds = hazard_raster.bounds
    ear_bounds = ear_raster.bounds
    hazard_transform = hazard_raster.transform
    ear_transform = ear_raster.transform
    hazard_raster_profile = hazard_raster.profile
    ear_raster_profile = ear_raster.profile

# Calculate intersection bounds
intersection_bounds = (
    max(ear_bounds.left, hazard_bounds.left),
    max(ear_bounds.bottom, hazard_bounds.bottom),
    min(ear_bounds.right, hazard_bounds.right),
    min(ear_bounds.top, hazard_bounds.top)
)
hazard_window = hazard_raster.window(*intersection_bounds)
hazard_row_off = int(hazard_window.row_off)
hazard_col_off = int(hazard_window.col_off)
hazard_height = int(hazard_window.height)
hazard_width = int(hazard_window.width)
clipped_hazard_raster = hazard_raster_data[hazard_row_off:(hazard_row_off + hazard_height), hazard_col_off:(hazard_col_off + hazard_width)]

ear_window = ear_raster.window(*intersection_bounds)
ear_row_off = int(ear_window.row_off)
ear_col_off = int(ear_window.col_off)
ear_height = int(ear_window.height)
ear_width = int(ear_window.width)
clipped_ear_raster = ear_raster_data[ear_row_off:(ear_row_off + ear_height), ear_col_off:(ear_col_off + ear_width)]

# ear_x_resolution = ear_raster.transform[0]
# ear_y_resolution = ear_raster.transform[4]  
ear_x_origin = ear_raster.transform[2]+(ear_col_off*final_x_res)
ear_y_origin = ear_raster.transform[5]+(ear_row_off*final_y_res)

hazard_x_resolution = hazard_raster.transform[0]
hazard_y_resolution = hazard_raster.transform[4]  
hazard_x_origin = hazard_raster.transform[2]+(hazard_col_off*final_x_res)
hazard_y_origin = hazard_raster.transform[5]+(hazard_row_off*final_y_res)

print(hazard_width,hazard_height,ear_height,ear_width)
print(ear_x_origin,ear_y_origin)
print(hazard_x_origin,hazard_y_origin)
# final_haz_out_path="/home/rabina/Projects/sdss_conda/data/ear_raster/output/FL_600_test_ready.tif"
# final_ear_out_path="/home/rabina/Projects/sdss_conda/data/ear_raster/output/lc_romania_test_ready.tif"


new_ear_transform = rasterio.transform.Affine(
    final_x_res,0.0, ear_x_origin,0.0,final_y_res, ear_y_origin)
new_hazard_transform = rasterio.transform.Affine(
    final_x_res,0.0, ear_x_origin,0.0,final_y_res, ear_y_origin)
# new_hazard_transform = rasterio.transform.Affine(
#     final_x_res,0.0, hazard_x_origin,0.0,final_y_res, hazard_y_origin)
ear_meta = {
    'driver': 'GTiff',
    'dtype': ear_raster_data.dtype,
    'count': 1,  # Number of bands
    'height': ear_height,
    'width': ear_width,
    'crs': ear_raster.crs,  # Replace with your desired CRS
    'transform': new_ear_transform
}
hazard_meta = {
    'driver': 'GTiff',
    'dtype': hazard_raster_data.dtype,
    'count': 1,  # Number of bands
    'height': hazard_height,
    'width': hazard_width,
    'crs': hazard_raster.crs,  # Replace with your desired CRS
    'transform': new_hazard_transform
}

with rasterio.open(final_haz_out_path, 'w', **hazard_meta) as dst1:
    dst1.write(clipped_hazard_raster, 1)
with rasterio.open(final_ear_out_path, 'w', **ear_meta) as dst1:
    dst1.write(clipped_ear_raster, 1)




In [ ]:

# hazard_window = hazard_raster.window(*intersection_bounds)
# hazard_row_off = int(hazard_window.row_off)
# hazard_col_off = int(hazard_window.col_off)
# hazard_height = int(hazard_window.height)
# hazard_width = int(hazard_window.width)
# clipped_hazard_raster = hazard_raster_data[hazard_row_off:(hazard_row_off + hazard_height), hazard_col_off:(hazard_col_off + hazard_width)]

# ear_window = ear_raster.window(*intersection_bounds)
# ear_row_off = int(ear_window.row_off)
# ear_col_off = int(ear_window.col_off)
# ear_height = int(ear_window.height)
# ear_width = int(ear_window.width)
# clipped_ear_raster = ear_raster_data[ear_row_off:(ear_row_off + ear_height), ear_col_off:(ear_col_off + ear_width)]

# # ear_x_resolution = ear_raster.transform[0]
# # ear_y_resolution = ear_raster.transform[4]  
# ear_x_origin = ear_raster.transform[2]+(ear_col_off*final_x_res)
# ear_y_origin = ear_raster.transform[5]+(ear_row_off*final_y_res)

# hazard_x_resolution = hazard_raster.transform[0]
# hazard_y_resolution = hazard_raster.transform[4]  
# hazard_x_origin = hazard_raster.transform[2]+(hazard_col_off*final_x_res)
# hazard_y_origin = hazard_raster.transform[5]+(hazard_row_off*final_y_res)

# print(hazard_width,hazard_height,ear_height,ear_width)
# print(ear_x_origin,ear_y_origin)
# print(hazard_x_origin,hazard_y_origin)

# # 8147 6263 6263 8147
# # 100.0 -100.0 411974.38883173274 5402846.243662003
# # 100.0 -100.0 412041.0951211243 5402869.7196161095

In [ ]:
# # final_haz_out_path="/home/rabina/Projects/sdss_conda/data/ear_raster/output/FL_600_test_ready.tif"
# # final_ear_out_path="/home/rabina/Projects/sdss_conda/data/ear_raster/output/lc_romania_test_ready.tif"


# new_ear_transform = rasterio.transform.Affine(
#     final_x_res,0.0, ear_x_origin,0.0,final_y_res, ear_y_origin)
# new_hazard_transform = rasterio.transform.Affine(
#     final_x_res,0.0, ear_x_origin,0.0,final_y_res, ear_y_origin)
# # new_hazard_transform = rasterio.transform.Affine(
# #     final_x_res,0.0, hazard_x_origin,0.0,final_y_res, hazard_y_origin)
# ear_meta = {
#     'driver': 'GTiff',
#     'dtype': ear_raster_data.dtype,
#     'count': 1,  # Number of bands
#     'height': ear_height,
#     'width': ear_width,
#     'crs': ear_raster.crs,  # Replace with your desired CRS
#     'transform': new_ear_transform
# }
# hazard_meta = {
#     'driver': 'GTiff',
#     'dtype': hazard_raster_data.dtype,
#     'count': 1,  # Number of bands
#     'height': hazard_height,
#     'width': hazard_width,
#     'crs': hazard_raster.crs,  # Replace with your desired CRS
#     'transform': new_hazard_transform
# }

# with rasterio.open(final_haz_out_path, 'w', **hazard_meta) as dst1:
#     dst1.write(clipped_hazard_raster, 1)
# with rasterio.open(final_ear_out_path, 'w', **ear_meta) as dst1:
#     dst1.write(clipped_ear_raster, 1)


## Combine haz and ear raster


In [ ]:
hazard_input_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/FL_ROU_600_UTM_resampled_reclassified_ready.tif"
ear_input_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/LandCover_ROU_UTM_classified_resampled_ready.tif"
haz_ear_overlay_path="/home/rabina/Projects/sdss_conda/data/gridded_ear_exposure/output/ear_haz_overlay.tif"


In [ ]:
with rasterio.open(hazard_input_path) as hazard_raster, rasterio.open(ear_input_path) as ear_raster:
    hazard_raster_data = hazard_raster.read(1)  # Read the first band of raster 1
    ear_raster_data = ear_raster.read(1) 
    # profile = hazard_raster.profile
    profile = hazard_raster.profile

# Update the profile to indicate the number of bands in the output raster
# profile['count']=2
# print(profile)
profile.update(count=2)

with rasterio.open(haz_ear_overlay_path, 'w', **profile) as dst:
    dst.write(hazard_raster_data, 1)  # Write the first band from the first raster
    dst.write(ear_raster_data, 2)
    print(dst.profile)

## Exposure Calculation

In [ ]:
with rasterio.open(haz_ear_overlay_path) as ear_hazard_raster:
    # print(ear_hazard_raster.count)
    hazard_data = ear_hazard_raster.read(1)
    ear_data = ear_hazard_raster.read(2) 
unique_ear_pixel_values=np.unique(ear_data)
unique_hazard_pixel_values=np.unique(hazard_data)

print(unique_ear_pixel_values)
print(unique_hazard_pixel_values)

# Create a DataFrame to store the pivot table
pivot_data = pd.DataFrame(index=unique_ear_pixel_values, columns=unique_hazard_pixel_values, dtype=int)

# Fill the pivot table with counts
for hazard_value in unique_hazard_pixel_values:
    for ear_value in unique_ear_pixel_values:
        count = np.sum((hazard_data == hazard_value) & (ear_data == ear_value))
        pivot_data.at[ear_value, hazard_value] = count

# Display the pivot table
print(pivot_data)


Romania
haz=30.7 MB
ear=4.3 MB

ONLY EXPOSURE CALCULATION TIME= 8.7s

## Rough


In [ ]:
# haz_ear_overlay_path="/Users/rabina/Projects/sdss/sdss_conda/data/ear_raster/input/lc_romania_ready.tif"
# abc=rasterio.open(haz_ear_overlay_path)
# hazard_data = abc.read(1)
# unique_ear_pixel_values=np.unique(hazard_data)
# print(abc.count)
# print(list(unique_ear_pixel_values))
    # print(ear_hazard_raster.descriptions)
import numpy as np
np.unique(clipped_ear_raster) 